In [16]:
# !pip install selenium
# !apt-get update
# !apt install chromiun-chromedriver

In [17]:
from selenium import webdriver

In [18]:
chrome_options = webdriver.ChromeOptions()

In [19]:
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [21]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


WebDriverException: ignored

In [1]:
import pandas as pd
import time
from tqdm import tqdm

navegador.get("https://www.flashscore.com/")
navegador.maximize_window()
time.sleep(5)

#Fechando botao de cookies
try:
    button_cookies = navegador.find_element(By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')
    button_cookies.click()
except:
    pass
navegador.find_element(By.CSS_SELECTOR,'div.calendar__navigation--tomorrow').click()
time.sleep(3)

#Pegando id dos jogos
id_jogos = []
jogos = navegador.find_elements(By.CSS_SELECTOR,'div.event__match--scheduled')
for i in jogos:
    try:
        id_jogos.append(i.get_attribute("id"))
    except:
        pass
id_jogos = [i[4:] for i in id_jogos]

jogo = {'date':[],'time':[],'country':[],'league':[],'home':[],'away':[],'odd_home':[],'odd_draw':[],'odd_away':[]}
for link in tqdm(id_jogos, total=len(id_jogos)):
    navegador.get(f'https://www.flashscore.com/match/{link}/#/match-summary')
    time.sleep(2)

    ######### Adicionando informacoes basicas do jogo
    try:
        #Data
        data = navegador.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
        #Horario
        hora = navegador.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
        #Pais
        pais = navegador.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country').text.split(':')[0]
        #Liga
        liga = navegador.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country')
        liga = liga.find_element(By.CSS_SELECTOR,'a').text
        #home
        home = navegador.find_element(By.CSS_SELECTOR,'div.duelParticipant__home')
        home = home.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        #away
        away = navegador.find_element(By.CSS_SELECTOR,'div.duelParticipant__away')
        away = away.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        ######### Acessando Match Odds
        navegador.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/1x2-odds/full-time')
        time.sleep(2)
        celulas = navegador.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        odd_home = 0
        odd_draw = 0
        odd_away = 0
        if 'title="bet365"' in str(navegador.find_element(By.CSS_SELECTOR,'div.ui-table.oddsCell__odds')):
            for celula in celulas:
                bookie = celula.find_element(By.CSS_SELECTOR,'img.prematchLogo')
                bookie = bookie.get_attribute('title')
                if ((bookie == 'bet365') & (odd_home == 0)) | ((bookie == 'Betfair') & (odd_home == 0)):
                    odd_home = celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text
                    odd_draw = celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text 
                    odd_away = celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[2].text
                else:
                    pass
        else:
            for celula in celulas:
                odd_home = celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text
                odd_draw = celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text 
                odd_away = celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[2].text

    except:
        pass
    ###Aqui eu tento imprimir para confirmar se existem as variaveis
    try:
        print(data,hora,pais,liga,home,away,odd_home,odd_draw,odd_away) 
        jogo['date'].append(data)
        jogo['time'].append(hora)
        jogo['country'].append(pais)
        jogo['league'].append(liga)
        jogo['home'].append(home)
        jogo['away'].append(away)
        jogo['odd_home'].append(odd_home)
        jogo['odd_draw'].append(odd_draw)
        jogo['odd_away'].append(odd_away)
        
    except:
        pass
df = pd.DataFrame(jogo)
df.to_excel('flashscore_match_odds.xlsx')

ModuleNotFoundError: ignored